In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import itertools
from IPython.display import display, HTML
from sklearn import linear_model
from sklearn.model_selection import KFold
from ipywidgets import interact
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact
import ipywidgets as widgets 

from scipy import stats

In [4]:
#Importar base de datos
df = pd.read_csv("Car_sales1.csv", delimiter=";") 
print (df.head())

  Manufacturer    Model  Sales_in_thousands  __year_resale_value Vehicle_type  \
0        Acura  Integra              16.919               16.360    Passenger   
1        Acura       TL              39.384               19.875    Passenger   
2        Acura       RL               8.588               29.725    Passenger   
3         Audi       A4              20.397               22.255    Passenger   
4         Audi       A6              18.780               23.555    Passenger   

   Price_in_thousands  Engine_size  Horsepower  Wheelbase  Width  Length  \
0               21.50          1.8         140      101.2   67.3   172.4   
1               28.40          3.2         225      108.1   70.3   192.9   
2               42.00          3.5         210      114.6   71.4   196.6   
3               23.99          1.8         150      102.6   68.2   178.0   
4               33.95          2.8         200      108.7   76.1   192.0   

   Curb_weight  Fuel_capacity  Fuel_efficiency  
0      

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         117 non-null    object 
 1   Model                117 non-null    object 
 2   Sales_in_thousands   117 non-null    float64
 3   __year_resale_value  117 non-null    float64
 4   Vehicle_type         117 non-null    object 
 5   Price_in_thousands   117 non-null    float64
 6   Engine_size          117 non-null    float64
 7   Horsepower           117 non-null    int64  
 8   Wheelbase            117 non-null    float64
 9   Width                117 non-null    float64
 10  Length               117 non-null    float64
 11  Curb_weight          117 non-null    float64
 12  Fuel_capacity        117 non-null    float64
 13  Fuel_efficiency      117 non-null    int64  
dtypes: float64(9), int64(2), object(3)
memory usage: 12.9+ KB


In [6]:
df= pd.get_dummies(df)

df = df.astype(int)

df.head()

,Sales_in_thousands,__year_resale_value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,...,Model_Tacoma,Model_Taurus,Model_Town car,Model_Villager,Model_Viper,Model_Voyager,Model_Windstar,Model_Wrangler,Vehicle_type_Car,Vehicle_type_Passenger
0,16,16,21,1,140,101,67,172,2,13,...,0,0,0,0,0,0,0,0,0,1
1,39,19,28,3,225,108,70,192,3,17,...,0,0,0,0,0,0,0,0,0,1
2,8,29,42,3,210,114,71,196,3,18,...,0,0,0,0,0,0,0,0,0,1
3,20,22,23,1,150,102,68,178,2,16,...,0,0,0,0,0,0,0,0,0,1
4,18,23,33,2,200,108,76,192,3,18,...,0,0,0,0,0,0,0,0,0,1


In [7]:
x = df[['Price_in_thousands', 'Engine_size','Manufacturer_Audi', 'Vehicle_type_Car', 'Manufacturer_BMW', 'Manufacturer_Chevrolet', 'Manufacturer_Dodge',
            'Manufacturer_Ford', 'Manufacturer_Honda', 'Manufacturer_Jeep',
            'Manufacturer_Mercedes-B', 'Manufacturer_Mitsubishi', 'Manufacturer_Nissan', 'Manufacturer_Plymouth',
            'Manufacturer_Porsche', 'Manufacturer_Toyota', 'Manufacturer_Acura', 'Manufacturer_Cadillac', 'Model_3000GT', 'Model_328i',
            'Model_Voyager', 'Model_TL', 'Model_Sentra', 'Model_Sable', 'Model_SL-Class', 'Model_SL', 'Model_Ranger', 'Model_Ram Wagon', 'Model_Ram Pickup',
            'Model_Prizm', 'Model_Passat', 'Model_Neon', 'Model_Mirage', 'Model_Maxima', 'Model_Malibu', 'Model_LeSabre', 'Model_Jetta',
            'Model_Integra' , 'Model_Grand Prix', 'Model_Grand Marquis', 'Model_Grand Cherokee', 'Model_Grand Am', 'Model_Galant', 'Model_F-Series',
            'Model_Explorer', 'Model_Elantra', 'Model_Caravan', 'Model_Camry'
            
]]
eps = np.random.randn(len(x)) 

In [9]:
y_np = df['Sales_in_thousands']
plt.figure(figsize=(10,7))


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

In [11]:
def get_models(k, X, y):
    """
    Fit all possible models that contain exactly k predictors.
    """
    
    # Fit all models accumulating Residual Sum of Squares (RSS)
    models = []
    for X_label in x:
        # Parse patsy formula
        X_smf = ' + '.join(X_label)
        f     = '{} ~ {}'.format(y.columns[0], X_smf)
        # Fit model
        model = smf.ols(formula=f, data=pd.concat([X, y], axis=1)).fit()
        # Return results
        models += [(f, model)]
    return models

def min_rss(statsmodels):
    """Return model with lowest Residual Sum of Squares (RSS)"""
    return sorted(statsmodels, key=lambda tup: tup[1].ssr)[0]


def max_adjr2(statsmodels):
    """Return model with max R-squared"""
    return sorted(statsmodels, reverse=True, key=lambda tup: tup[1].rsquared_adj)[0]

def min_bic(statsmodels):
    """Return model with min Bayes' Information Criteria"""
    return sorted(statsmodels, reverse=False, key=lambda tup: tup[1].bic)[0]

def min_aic(statsmodels):
    """Return model with min Akaike's Information Criteria"""
    return sorted(statsmodels, reverse=False, key=lambda tup: tup[1].aic)[0]